In [1]:
import pandas
import RDF
from curation_common import *

In [2]:
server = ENCODED('www.encodeproject.org')
server.load_netrc()

In [3]:
single_cell_search = server.get_json("https://www.encodeproject.org/search/?type=experiment&lab.title=Barbara+Wold,+Caltech&award.rfa=ENCODE3&assay_term_name=single+cell+isolation+followed+by+RNA-seq&status=submitted&status=started")

In [4]:
model = get_model(use_contexts=False)

In [5]:
for i, exp in enumerate(single_cell_search['@graph']):
    exp_id = exp['@id']
    print(exp_id, i, len(single_cell_search['@graph']))
    load_jsonld_into_model(model, server.get_jsonld(exp_id))

/experiments/ENCSR734SOQ/ 0 52
/experiments/ENCSR909YEQ/ 1 52
/experiments/ENCSR298YHA/ 2 52
/experiments/ENCSR220HFQ/ 3 52
/experiments/ENCSR650MMK/ 4 52
/experiments/ENCSR953TKB/ 5 52
/experiments/ENCSR295UJP/ 6 52
/experiments/ENCSR323FHJ/ 7 52
/experiments/ENCSR195BCE/ 8 52
/experiments/ENCSR782WIL/ 9 52
/experiments/ENCSR138VRG/ 10 52
/experiments/ENCSR390KQJ/ 11 52
/experiments/ENCSR427FOY/ 12 52
/experiments/ENCSR846FYI/ 13 52
/experiments/ENCSR927IOO/ 14 52
/experiments/ENCSR739YUO/ 15 52
/experiments/ENCSR100KRK/ 16 52
/experiments/ENCSR198EIH/ 17 52
/experiments/ENCSR746LUP/ 18 52
/experiments/ENCSR637NLF/ 19 52
/experiments/ENCSR320PXZ/ 20 52
/experiments/ENCSR027CXR/ 21 52
/experiments/ENCSR662UPQ/ 22 52
/experiments/ENCSR521YLY/ 23 52
/experiments/ENCSR559QTE/ 24 52
/experiments/ENCSR527DQV/ 25 52
/experiments/ENCSR167AWS/ 26 52
/experiments/ENCSR708EHJ/ 27 52
/experiments/ENCSR474NEW/ 28 52
/experiments/ENCSR282DYQ/ 29 52
/experiments/ENCSR367DUL/ 30 52
/experiments/ENCSR

In [6]:
%%sparql -m model -c -o lib_to_alias
select ?lib ?alias
where {
    ?exp a experiment:experiment ;
         experiment:replicates ?rep .
    ?rep replicate:library ?lib .
    ?lib library:aliases ?alias .
}

Found 92 rows.


In [7]:
for i, row in enumerate(lib_to_alias):
    lib_url = str(row['alias']).replace('barbara-wold:', 
                                       'https://jumpgate.caltech.edu/library/')
    lib_url += '/'
    lib_id = str(row['alias']).replace('barbara-wold:','')
    load_into_model(model, 'rdfa', lib_url)
    print(i, len(lib_to_alias))
    s = RDF.Statement(
        row['lib'], 
        RDF.Node(RDF.Uri("http://jumpgate.caltech.edu/wiki/LibraryOntology#library")),
        RDF.Node(RDF.Uri(lib_url)))
    model.add_statement(s)
    s = RDF.Statement(
        row['lib'], 
        RDF.Node(RDF.Uri("http://jumpgate.caltech.edu/wiki/LibraryOntology#library")),
        RDF.Node(RDF.Uri(lib_url)))
    model.add_statement(s)
    s = RDF.Statement(
        RDF.Node(RDF.Uri(lib_url)),
        RDF.Node(RDF.Uri("http://jumpgate.caltech.edu/wiki/LibraryOntology#experiment")),
        row['lib'])
    model.add_statement(s)

0 92
1 92
2 92
3 92
4 92
5 92
6 92
7 92
8 92
9 92
10 92
11 92
12 92
13 92
14 92
15 92
16 92
17 92
18 92
19 92
20 92
21 92
22 92
23 92
24 92
25 92
26 92
27 92
28 92
29 92
30 92
31 92
32 92
33 92
34 92
35 92
36 92
37 92
38 92
39 92
40 92
41 92
42 92
43 92
44 92
45 92
46 92
47 92
48 92
49 92
50 92
51 92
52 92
53 92
54 92
55 92
56 92
57 92
58 92
59 92
60 92
61 92
62 92
63 92
64 92
65 92
66 92
67 92
68 92
69 92
70 92
71 92
72 92
73 92
74 92
75 92
76 92
77 92
78 92
79 92
80 92
81 92
82 92
83 92
84 92
85 92
86 92
87 92
88 92
89 92
90 92
91 92


In [15]:
%%sparql -m model -c -o encode_names
select ?exp ?description ?jumpgate ?library_id ?name
where {
    ?exp a experiment:experiment ;
         rdf:description ?description ;
         experiment:replicates ?rep .
       
    ?rep replicate:library ?lib .
    ?lib htsw:library ?jumpgate .
    ?jumpgate htsw:name ?name ;
              htsw:library_id ?library_id . 
}

Found 92 rows.


In [17]:
def to_df(results, columns=[]):
    stuff = []
    for row in results:
        stuff.append({ key: str(row[key]) for key in row})
            
    return pandas.DataFrame(stuff, columns=columns)


In [18]:
dcc_frame = to_df(encode_names, ['exp', 'description', 'library_id', 'name'])

In [20]:
dcc_frame.to_csv('/tmp/single-cell.csv')